In [ ]:
import time
import threading

# Start time of script execution 
ExecuteStart = system.date.now()
print "Execute Script Start Time: ", ExecuteStart

# Define the 20-minute timer (1200 seconds)
MAX_EXECUTION_TIME = 1200  # 20 minutes in seconds
execution_timed_out = [False]  # List to use mutable reference in nested functions

def timer_function():
    time.sleep(MAX_EXECUTION_TIME)
    if not all(insert_statuses):  # Only mark timeout if inserts aren't completed
        print "Execution timed out after 20 minutes."
        execution_timed_out[0] = True

# Start the timer thread
timeout_thread = threading.Thread(target=timer_function)
timeout_thread.start()

# Define start and end dates
sDate = system.date.getDate(2024, 1, 11)
eDate = system.date.getDate(2024, 1, 12)
startDate = system.date.format(sDate, "yyyy-MM-dd HH:mm:ss")
endDate = system.date.format(eDate, "yyyy-MM-dd HH:mm:ss")
numDays = system.date.daysBetween(sDate, eDate)
timeout = numDays * 1000 * 24 * 12
returnSize = numDays * 24 * 60

print "Start Date:", startDate
print "End Date:", endDate
print "Number of Days:", numDays
print "Script Timeout:", timeout
print "Return Size:", returnSize

paths = ["[hist/iqaluit scada system - wtp:wtp]analog/fit_5001/val"]
aggregationModes = ["Average"]

data = system.tag.queryTagHistory(paths=paths, startDate=startDate, endDate=endDate, returnSize=returnSize, aggregationModes=aggregationModes, returnFormat='Tall', timeout=timeout)
columnNames = list(data.getColumnNames())
print "Column Names:", columnNames
pyDataSet = system.dataset.toPyDataSet(data)
rowCount = pyDataSet.getRowCount()
print "Number of rows in dataset:", rowCount

insert_statuses = [False] * rowCount  # Tracks completion of each row

def increment_completed_inserts(row_index):
    insert_statuses[row_index] = True

def insertRowAsync(row, row_index, attempt=1, max_attempts=3):
    if execution_timed_out[0]:  # Stop insertion if timeout occurred
        print "Insert stopped due to timeout."
        return
    try:
        tag_name = row[0]
        value = row[1]
        T_stamp = row[3]
        value = round(value, 2) if isinstance(value, (float, int)) else value
        if value is not None:
            query = """
                INSERT INTO records (T_stamp, tag_name, value)
                VALUES (?, ?, ?)
                ON CONFLICT (T_stamp, tag_name)
                DO UPDATE SET value = EXCLUDED.value;
            """
            system.db.runPrepUpdate(query, [T_stamp, tag_name, value], "IQ_Report")
            print "Row inserted with T_stamp:", T_stamp
        else:
            print "Skipping row due to None value:", row
        increment_completed_inserts(row_index)
    except Exception:
        if attempt < max_attempts:
            print "Error inserting row with T_stamp:", T_stamp, "| Attempt:", attempt, "| Retrying..."
            system.util.invokeLater(lambda: insertRowAsync(row, row_index, attempt + 1, max_attempts), 1000)
        else:
            print "Failed to insert row with T_stamp after", max_attempts, "attempts:", T_stamp
            increment_completed_inserts(row_index)

sleep_time = 200
for idx, row in enumerate(pyDataSet):
    system.util.invokeLater(lambda r=row, i=idx: system.util.invokeAsynchronous(lambda: insertRowAsync(r, i)), idx * sleep_time)

def wait_for_inserts():
    print "Waiting for all inserts to complete or timeout..."
    while not all(insert_statuses):
        if execution_timed_out[0]:  # Exit the loop if timeout occurs
            print "Exiting wait due to timeout."
            return
        print "Completed inserts:", sum(insert_statuses), "out of", rowCount
        time.sleep(1)

wait_for_inserts()

ExecuteEnd = system.date.now()
print "Execute Script End Time:", ExecuteEnd
print "It took", system.date.secondsBetween(ExecuteStart, ExecuteEnd), "seconds to execute."
